{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Schwab API: SPY & S&P 500 Options and Price Analysis"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Data Fetching\n",
    "\n",
    "Import the `schwab_data_analyzer` module and run the main data fetching function. This will connect to the Schwab API, retrieve data for SPY, S&P 500 Index ($SPX.X), and attempt for E-mini S&P 500 Futures (/ES). All retrieved raw data will be saved as JSON files in the `schwab_api_output_options` directory."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import importlib.util\n",
    "import os\n",
    "import json\n",
    "import pandas as pd\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "\n",
    "# --- Load the schwab_data_analyzer.py module --- \n",
    "# Assuming 'schwab_data_analyzer.py' is in the same directory as this notebook.\n",
    "module_name = 'sda' # Alias for the module\n",
    "file_path = 'schwab_data_analyzer.py'\n",
    "\n",
    "spec = importlib.util.spec_from_file_location(module_name, file_path)\n",
    "if spec is None:\n",
    "    raise ImportError(f\"Could not load spec for module from file: {file_path}. Ensure the file exists.\")\n",
    "sda = importlib.util.module_from_spec(spec)\n",
    "spec.loader.exec_module(sda)\n",
    "print(\"Module 'schwab_data_analyzer.py' loaded as 'sda'.\")\n",
    "\n",
    "# Configure Plotly for offline use in Jupyter Lab/Notebook if needed\n",
    "# from plotly.offline import init_notebook_mode\n",
    "# init_notebook_mode(connected=True) \n",
    "\n",
    "# --- Execute Data Fetching --- \n",
    "client = sda._initialize_client() # Initialize client via the module's function\n",
    "if client:\n",
    "    print(\"\\nStarting data fetching process...\")\n",
    "    sda.fetch_all_spy_and_sp500_data(client)\n",
    "    print(\"\\nData fetching attempts complete. Check 'schwab_api_output_options' directory.\")\n",
    "else:\n",
    "    print(\"Client initialization failed. Cannot fetch data. Check .env file and API credentials.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load and Process Price History Data\n",
    "\n",
    "Load the saved price history JSON files and process them to get DataFrames with price changes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_price_history_json(filename_prefix, symbol_name):\n",
    "    filepath = os.path.join(sda.OUTPUT_DIRECTORY, f\"{filename_prefix}_price_history.json\")\n",
    "    try:\n",
    "        with open(filepath, 'r') as f:\n",
    "            history_json = json.load(f)\n",
    "        print(f\"Successfully loaded: {filepath}\")\n",
    "        # Use the helper from the module to process this JSON\n",
    "        if sda: # Check if module is loaded\n",
    "             return sda.get_underlying_price_changes_df(history_json, symbol_name)\n",
    "        else:\n",
    "            print(\"Module 'sda' not loaded. Cannot process price history.\")\n",
    "            return None\n",
    "    except FileNotFoundError:\n",
    "        print(f\"ERROR: Price history file not found: {filepath}\")\n",
    "        return None\n",
    "    except Exception as e:\n",
    "        print(f\"ERROR loading or processing {filepath}: {e}\")\n",
    "        return None\n",
    "\n",
    "spy_prices_df = load_price_history_json(\"SPY\", \"SPY\")\n",
    "spx_prices_df = load_price_history_json(\"SPX_Index\", \"SPX_Index\")\n",
    "es_prices_df = load_price_history_json(\"ES_Future\", \"ES_Future\") # Attempt for /ES or ES\n",
    "\n",
    "if spy_prices_df is not None:\n",
    "    print(\"\\nSPY Price Data Sample:\")\n",
    "    display(spy_prices_df.head(3))\n",
    "if spx_prices_df is not None:\n",
    "    print(\"\\nSPX Index Price Data Sample:\")\n",
    "    display(spx_prices_df.head(3))\n",
    "if es_prices_df is not None:\n",
    "    print(\"\\nES Future Price Data Sample:\")\n",
    "    display(es_prices_df.head(3))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Plot Underlying Prices and Price Changes"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.1 SPY Price and Price Changes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if spy_prices_df is not None and not spy_prices_df.empty:\n",
    "    fig_spy_close = px.line(spy_prices_df, y='SPY_close', title='SPY Closing Price Over Time')\n",
    "    fig_spy_close.update_layout(yaxis_title='Price (USD)')\n",
    "    fig_spy_close.show()\n",
    "    \n",
    "    fig_spy_change = px.line(spy_prices_df, y='SPY_price_change', title='SPY Daily Price Change Over Time')\n",
    "    fig_spy_change.add_hline(y=0, line_dash=\"dash\", line_color=\"gray\")\n",
    "    fig_spy_change.update_layout(yaxis_title='Price Change (USD)')\n",
    "    fig_spy_change.show()\n",
    "    \n",
    "    fig_spy_pct_change = px.line(spy_prices_df, y='SPY_pct_change', title='SPY Daily Percentage Price Change Over Time')\n",
    "    fig_spy_pct_change.add_hline(y=0, line_dash=\"dash\", line_color=\"gray\")\n",
    "    fig_spy_pct_change.update_layout(yaxis_title='Percentage Change (%)')\n",
    "    fig_spy_pct_change.show()\n",
    "else:\n",
    "    print(\"SPY price data not available for plotting.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.2 S&P 500 Index ($SPX.X) Price and Price Changes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if spx_prices_df is not None and not spx_prices_df.empty:\n",
    "    fig_spx_close = px.line(spx_prices_df, y='SPX_Index_close', title='S&P 500 Index ($SPX.X) Closing Level Over Time')\n",
    "    fig_spx_close.update_layout(yaxis_title='Index Level')\n",
    "    fig_spx_close.show()\n",
    "    \n",
    "    fig_spx_change = px.line(spx_prices_df, y='SPX_Index_price_change', title='S&P 500 Index ($SPX.X) Daily Level Change Over Time')\n",
    "    fig_spx_change.add_hline(y=0, line_dash=\"dash\", line_color=\"gray\")\n",
    "    fig_spx_change.update_layout(yaxis_title='Level Change')\n",
    "    fig_spx_change.show()\n",
    "else:\n",
    "    print(\"S&P 500 Index ($SPX.X) price data not available for plotting.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.3 E-mini S&P 500 Future (/ES) Price and Price Changes (if available)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if es_prices_df is not None and not es_prices_df.empty:\n",
    "    fig_es_close = px.line(es_prices_df, y='ES_Future_close', title='E-mini S&P 500 Future Closing Price Over Time')\n",
    "    fig_es_close.update_layout(yaxis_title='Price (USD)')\n",
    "    fig_es_close.show()\n",
    "    \n",
    "    fig_es_change = px.line(es_prices_df, y='ES_Future_price_change', title='E-mini S&P 500 Future Daily Price Change Over Time')\n",
    "    fig_es_change.add_hline(y=0, line_dash=\"dash\", line_color=\"gray\")\n",
    "    fig_es_change.update_layout(yaxis_title='Price Change (USD)')\n",
    "    fig_es_change.show()\n",
    "else:\n",
    "    print(\"E-mini S&P 500 Future (/ES) price data not available for plotting. This might be due to symbol issues or API limitations for continuous futures.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Load and Display Options Data\n",
    "\n",
    "Load some of the saved options chain data and display key information.\n",
    "Plotting options data can be complex (e.g., volatility smiles, payoff diagrams). Here, we'll focus on displaying a sample."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def display_sample_options_data(filename_prefix, underlying_symbol):\n",
    "    print(f\"\\n--- Sample Options Data for {underlying_symbol} ({filename_prefix}) ---\")\n",
    "    # First, try to load the expiration chain to find an expiration date\n",
    "    exp_chain_filepath = os.path.join(sda.OUTPUT_DIRECTORY, f\"{filename_prefix}_option_exp_chain.json\")\n",
    "    exp_date_to_load = None\n",
    "    try:\n",
    "        with open(exp_chain_filepath, 'r') as f:\n",
    "            exp_chain_json = json.load(f)\n",
    "        if exp_chain_json.get('status') == 'SUCCESS' and exp_chain_json.get('expirationList'):\n",
    "            # Take the first available expiration date from the saved chain as an example\n",
    "            sorted_expirations = sorted(exp_chain_json['expirationList'], key=lambda x: x['expirationDate'])\n",
    "            if sorted_expirations:\n",
    "                 exp_date_to_load = sorted_expirations[0]['expirationDate'].split('T')[0].replace('-', '')\n",
    "    except FileNotFoundError:\n",
    "        print(f\"Expiration chain file not found: {exp_chain_filepath}\")\n",
    "    except Exception as e:\n",
    "        print(f\"Error loading expiration chain {exp_chain_filepath}: {e}\")\n",
    "\n",
    "    if not exp_date_to_load:\n",
    "        print(f\"Could not determine an example expiration date for {underlying_symbol}. Cannot load sample option chain.\")\n",
    "        return\n",
    "\n",
    "    option_chain_filepath = os.path.join(sda.OUTPUT_DIRECTORY, f\"{filename_prefix}_option_chain_{exp_date_to_load}.json\")\n",
    "    try:\n",
    "        with open(option_chain_filepath, 'r') as f:\n",
    "            options_json = json.load(f)\n",
    "        print(f\"Successfully loaded: {option_chain_filepath}\")\n",
    "        \n",
    "        if options_json.get('status') == 'SUCCESS':\n",
    "            print(f\"Underlying Price: {options_json.get('underlyingPrice', 'N/A')}\")\n",
    "            print(f\"Interest Rate: {options_json.get('interestRate', 'N/A')}, Volatility: {options_json.get('volatility', 'N/A')}\")\n",
    "            \n",
    "            # Display a few call options\n",
    "            if 'callExpDateMap' in options_json and options_json['callExpDateMap']:\n",
    "                # The structure is callExpDateMap -> { \"YYYY-MM-DD (exp)\": { \"STRIKE\": [options_list] } }\n",
    "                # We fetched for a specific date, so there should be one key in callExpDateMap\n",
    "                date_key = list(options_json['callExpDateMap'].keys())[0]\n",
    "                calls = options_json['callExpDateMap'][date_key]\n",
    "                print(\"\\nSample Call Options:\")\n",
    "                count = 0\n",
    "                for strike, details_list in list(calls.items())[:3]: # First 3 strikes\n",
    "                    for details in details_list[:1]: # First option for that strike\n",
    "                        print(f\"  Strike: {strike}, Last: {details.get('last')}, Bid: {details.get('bid')}, Ask: {details.get('ask')}, IV: {details.get('volatility', 'N/A')}, Delta: {details.get('delta', 'N/A')}\")\n",
    "                        count += 1\n",
    "                if count == 0: print(\"  No call options found in this sample.\")\n",
    "\n",
    "            # Display a few put options\n",
    "            if 'putExpDateMap' in options_json and options_json['putExpDateMap']:\n",
    "                date_key = list(options_json['putExpDateMap'].keys())[0]\n",
    "                puts = options_json['putExpDateMap'][date_key]\n",
    "                print(\"\\nSample Put Options:\")\n",
    "                count = 0\n",
    "                for strike, details_list in list(puts.items())[:3]: # First 3 strikes\n",
    "                    for details in details_list[:1]: # First option for that strike\n",
    "                        print(f\"  Strike: {strike}, Last: {details.get('last')}, Bid: {details.get('bid')}, Ask: {details.get('ask')}, IV: {details.get('volatility', 'N/A')}, Delta: {details.get('delta', 'N/A')}\")\n",
    "                        count += 1\n",
    "                if count == 0: print(\"  No put options found in this sample.\")\n",
    "        else:\n",
    "            print(f\"Option chain data for {underlying_symbol} ({exp_date_to_load}) indicates failure or no data: {options_json.get('error', 'Unknown issue')}\")\n",
    "\n",
    "    except FileNotFoundError:\n",
    "        print(f\"ERROR: Option chain file not found: {option_chain_filepath}\")\n",
    "    except Exception as e:\n",
    "        print(f\"ERROR loading or displaying options data from {option_chain_filepath}: {e}\")\n",
    "\n",
    "if client: # Ensure client was initialized before trying to display options\n",
    "    display_sample_options_data(\"SPY\", \"SPY\")\n",
    "    display_sample_options_data(\"SPX_Index\", \"S&P 500 Index ($SPX.X)\")\n",
    "    display_sample_options_data(\"ES_Future_Options\", \"E-mini S&P 500 Future (/ES)\") # Attempt for /ES or ES\n",
    "else:\n",
    "    print(\"Schwab client not initialized. Skipping options data display.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--- End of Notebook ---"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12" 
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
